<a href="https://colab.research.google.com/github/JNarimatsu/Python-para-IA---Primeiro-chatbot-Asimov/blob/main/Primeiro_chatobot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install langchain==0.3.0 #Instalando langchain (LangChain é uma biblioteca projetada para simplificar a construção de aplicações que usam modelos de linguagem avançados, como GPT.)
!pip install langchain-groq==0.2.0 #Instalando o groq para utilizanção da API KEY
!pip install langchain-community==0.3.0 #Acesso sites
!pip install youtube_transcript_api==0.6.2 #Acesso ao youtube
!pip install pypdf==5.0.0 #Acesso a PDF
!pip install requests


In [14]:
#Imposts
import os
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import YoutubeLoader
from langchain_community.document_loaders import PyPDFLoader
import requests
import tempfile

In [15]:
#Criando o Chatbot
api_key = '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'
os.environ['GROQ_API_KEY'] = api_key
chat = ChatGroq(model="llama-3.3-70b-versatile")


def resposta_bot(mensagens, documento):
  system_message = '''Você é um assistente amigável chamado Asimo.
  Você utiliza as seguintes informações para formular as suas repostas: {informacoes}'''
  mensagens_modelo = [('system', system_message)]
  mensagens_modelo += mensagens
  template = ChatPromptTemplate.from_messages(mensagens_modelo)
  chain = template | chat
  return chain.invoke({'informacoes': documento}).content

In [16]:
#Criando funções para receber documents loaders
def carrega_site():
  url_site = input('Digite a url do site: ')
  loader = WebBaseLoader(url_site)
  lista_documentos = loader.load()
  documento = ''
  for doc in lista_documentos:
    documento = documento + doc.page_content
    return documento


def carrega_pdf():
  url_pdf = input('Digite a URL do PDF: ')

  # Faz o download do PDF
  response = requests.get(url_pdf)
  if response.status_code != 200:
    print("Erro ao baixar o PDF. Verifique a URL.")
    return ""

  # Salva temporariamente
  with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
    tmp_file.write(response.content)
    caminho_temp = tmp_file.name

  # Lê com PyPDFLoader
  loader = PyPDFLoader(caminho_temp)
  lista_documentos = loader.load()

  documento = ''
  for doc in lista_documentos:
    documento += doc.page_content + '\n'
  return documento


def carrega_youtube():
  url_youtube = input('Digite a url do vídeo: ')
  loader = YoutubeLoader.from_youtube_url(url_youtube, language=['pt'])
  lista_documentos = loader.load()
  documento = ''
  for doc in lista_documentos:
    documento = documento + doc.page_content
  return documento

In [20]:
#Loops de selecão e de conversa
print('Bem-vindo ao AsimoBot')


texto_selecao = ''' Digite 1 se você quiser verificar um site
Digite 2 se você quiser verificar um PDF
Digite 3 se você quiser verificar um vídeo do youtube
'''
while True:
  selecao = input(texto_selecao)
  if selecao == '1':
    documento = carrega_site()
    break
  if selecao == '2':
    documento = carrega_pdf()
    break
  if selecao == '3':
    documento = carrega_youtube()
    break
  else:
    print('Opção inválida, digite uma opção válida!')

mensagens = []
while True:
  pergunta = input('Usuário: ')
  if pergunta.lower() == 'x':
    break
  mensagens.append(('user', pergunta))
  resposta = resposta_bot(mensagens, documento)
  mensagens.append(('assistant', resposta))
  print(f'Bot: {resposta}')
print('Muito Obrigada por usar o AsimoBot')
print(mensagens)

Bem-vindo ao AsimoBot
 Digite 1 se você quiser verificar um site
Digite 2 se você quiser verificar um PDF
Digite 3 se você quiser verificar um vídeo do youtube
3
Digite a url do vídeo: https://www.youtube.com/watch?v=4MViYMwyEHw
Usuário: do que se trata
Bot: Essa parece ser uma discussão sobre uma polêmica envolvendo dois bilionários, Bill Gates e Elon Musk. A polêmica gira em torno de cortes feitos pelo governo dos Estados Unidos na ajuda humanitária internacional, especificamente no programa USAID (Agência dos Estados Unidos para o Desenvolvimento Internacional), que é responsável por fornecer ajuda humanitária e de desenvolvimento em todo o mundo.

Bill Gates, que é conhecido por sua filantropia e trabalho em causa de caridade, especialmente na área de saúde, fez declarações criticando Elon Musk, que é o homem mais rico do mundo e que tem sido acusado de ter influência no governo dos Estados Unidos. Gates acusou Musk de estar "matando as crianças mais pobres do mundo" devido aos cor